In [4]:
# tf-idf preprocess
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from scipy import sparse
train_merged=pd.read_csv('train_merged.csv')
df=train_merged
v = TfidfVectorizer(max_df=50,lowercase=False,tokenizer=lambda x:x)
cur=df.groupby("user_id")["creative_id"].apply(list)
# cur.astype(int)
display(cur)
v_fit = v.fit_transform(cur)
display(v_fit)
sparse.save_npz('v_fit.npz', v_fit)

user_id
1         [71691, 90171, 122032, 209778, 821396, 877468,...
2         [13069, 15558, 22013, 39714, 63441, 96192, 155...
3         [66009, 72533, 392052, 593522, 599128, 661347,...
4         [31070, 39588, 72989, 215041, 574787, 589886, ...
5         [24333, 43235, 75011, 296145, 350759, 795508, ...
                                ...                        
899996    [92193, 114074, 654526, 931537, 939409, 145880...
899997    [7400, 24333, 30210, 60330, 69204, 103918, 103...
899998    [71752, 648402, 1251649, 1370659, 1779124, 230...
899999    [12838, 12838, 12838, 12838, 122834, 279342, 3...
900000    [234063, 285502, 423617, 1224107, 1405713, 180...
Name: creative_id, Length: 900000, dtype: object

<900000x2414321 sparse matrix of type '<class 'numpy.float64'>'
	with 9965541 stored elements in Compressed Sparse Row format>

In [1]:
# cross validation for gender
%reset -f
import pandas as pd
from scipy import sparse
user_info=pd.read_csv('user_info.csv')
user_info.astype(int)
display(user_info)
agg=pd.read_csv('agg.csv')
# agg.astype(int)
display(agg)
v_fit=sparse.load_npz('v_fit.npz')
num_train=int(user_info.shape[0]*0.8)
y_train=user_info["gender"].iloc[:num_train]
y_test=user_info["gender"].iloc[num_train:]

# X_train=sparse.csr_matrix(v_fit[:num_train,:],dtype=float).tocsr()
# X_test=sparse.csr_matrix(v_fit[num_train:,:],dtype=float).tocsr()

X_train=sparse.csr_matrix(agg.iloc[:num_train])
X_test=sparse.csr_matrix(agg.iloc[num_train:])
X_train=sparse.hstack((v_fit[:num_train,:],X_train),dtype=float).tocsr()
X_test=sparse.hstack((v_fit[num_train:,:],X_test),dtype=float).tocsr()

import lightgbm as lgb
train_data = lgb.Dataset(X_train,y_train)
train_data.save_binary('CountVectorizer_train_gender.bin')
# test_data=lgb.Dataset(X_test,y_test)
# test_data.save_binary('CountVectorizer_test_gender.bin')

,user_id,age,gender
0,1,4,0
1,2,10,0
2,3,7,1
3,4,5,0
4,5,4,0
...,...,...,...
899995,899996,5,0
899996,899997,3,1
899997,899998,4,1
899998,899999,3,0


,click_times_mean,advertiser_id_nunique,advertiser_id_count,industry_nunique,industry_count,time_nunique,time_count,product_category_nunique,product_category_count,product_id_nunique,product_id_count,ad_id_nunique,ad_id_count
0,1.076923,12,13,9,13,10,13,3,13,6,13,12,13
1,1.022222,36,45,15,45,28,45,3,45,20,45,42,45
2,1.000000,28,30,8,30,23,30,6,30,17,30,30,30
3,1.000000,26,29,10,29,15,29,6,29,18,29,29,29
4,1.030303,30,33,18,33,26,33,4,33,7,33,33,33
...,...,...,...,...,...,...,...,...,...,...,...,...,...
899995,1.000000,12,14,5,14,12,14,3,14,5,14,13,14
899996,1.111111,13,18,10,18,14,18,4,18,10,18,17,18
899997,1.071429,9,14,5,14,10,14,4,14,5,14,14,14
899998,1.000000,16,22,14,22,17,22,7,22,5,22,18,22


In [ ]:
# cross validation for gender
%reset -f
import lightgbm as lgb
train_data = lgb.Dataset('CountVectorizer_train_gender.bin')
lgb_params = {'num_leaves': 2**7-1,
              'min_data_in_leaf': 25, 
              'objective':'binary',
              'max_depth': -1,
              'learning_rate': 0.1,
              'boosting': 'gbdt',
              'feature_fraction': 0.6,
              'bagging_fraction': 0.9,
              'bagging_seed': 11,
              'metric': 'auc',
              'seed':1024,
              'nthread':12,
             }
print("start training")
cv_results = lgb.cv(
    lgb_params, train_data, num_boost_round=1000, nfold=5, shuffle=True,
    early_stopping_rounds=50, verbose_eval=10, show_stdv=True)

start training
[10]	cv_agg's auc: 0.6231 + 0.00142673
[20]	cv_agg's auc: 0.628205 + 0.00127108
[30]	cv_agg's auc: 0.641034 + 0.000831938
[40]	cv_agg's auc: 0.648032 + 0.000781777
[50]	cv_agg's auc: 0.656094 + 0.00104904
[60]	cv_agg's auc: 0.66377 + 0.00129256
[70]	cv_agg's auc: 0.668926 + 0.00125677
[80]	cv_agg's auc: 0.673008 + 0.00128241
[90]	cv_agg's auc: 0.676582 + 0.00132133
[100]	cv_agg's auc: 0.679915 + 0.00122345
[110]	cv_agg's auc: 0.683032 + 0.00119218
[120]	cv_agg's auc: 0.685907 + 0.00122485
[130]	cv_agg's auc: 0.688598 + 0.00119378
[140]	cv_agg's auc: 0.691029 + 0.00108103
[150]	cv_agg's auc: 0.693345 + 0.00114
[160]	cv_agg's auc: 0.695466 + 0.00114003
[170]	cv_agg's auc: 0.697398 + 0.0011217
[180]	cv_agg's auc: 0.699246 + 0.00114483
[190]	cv_agg's auc: 0.700961 + 0.00117523


In [1]:
# predict gender
%reset -f
import lightgbm as lgb
train_data = lgb.Dataset('CountVectorizer_train_gender.bin')
# test_data = lgb.Dataset('CountVectorizer_test_gender.bin')

lgb_params = {'num_leaves': 2**7-1,
              'min_data_in_leaf': 25, 
              'objective':'binary',
              'max_depth': -1,
              'learning_rate': 0.1,
              'boosting': 'gbdt',
              'feature_fraction': 0.6,
              'bagging_fraction': 0.9,
              'bagging_seed': 11,
              'metric': 'auc',
              'seed':1024,
              'nthread':12,
             }
print("start training")
model = lgb.LGBMClassifier(lgb_params, train_data,num_boost_round=200)
model=lgb.LGBMClassifier(boosting_type='gbdt',
                         objective='binary',
                         metrics='auc',
                         learning_rate=0.1,
                         n_estimators=200, 
                         max_depth=-1, 
                         num_leaves= 2**7-1,
                         feature_fraction= 0.6,
                         bagging_fraction=0.9,
                         nthread=12)
print("training completed!")

start training


LightGBMError: Cannot add validation data, since it has different bin mappers with training data

In [ ]:
# predict gender
%reset -f
import pandas as pd
from scipy import sparse
import lightgbm as lgb
from sklearn import metrics
user_info=pd.read_csv('user_info.csv')
user_info.astype(int)
# display(user_info)
agg=pd.read_csv('agg.csv')
# display(agg)
v_fit=sparse.load_npz('v_fit.npz')
num_train=int(user_info.shape[0]*0.8)
y_test=user_info["gender"].iloc[num_train:]
# display(y_test)
X_test=sparse.csr_matrix(agg.iloc[num_train:])
X_test=sparse.hstack((v_fit[num_train:,:],X_test),dtype=float).tocsr()
print("start predicting")
model=lgb.Booster(model_file='model.txt')
y_pred=model.predict(X_test)
y_pred=(y_pred>0.5)*1
print("acc:",metrics.accuracy_score(y_test,y_pred))
print("acc:",metrics.roc_auc_score(y_test,y_pred))
# submission=pd.DataFrame(result,columns=["gender"])
# submission["user_id"]=[i for i in range(num_train+1,user_info.shape[0]+1)]
# display(submission)

In [ ]:
# cross validation for age
%reset -f
# !pip install lightgbm
import lightgbm as lgb
import pandas as pd
from scipy import sparse
user_info=pd.read_csv('user_info.csv')
agg=pd.read_csv('agg.csv')
v_fit=sparse.load_npz('v_fit.npz')
num_train=int(user_info.shape[0]*0.8)
y_train=user_info["age"].iloc[:num_train]
y_test=user_info["age"].iloc[num_train:]
X_train=sparse.csr_matrix(agg.iloc[:num_train])
X_test=sparse.csr_matrix(agg.iloc[num_train:])
X_train=sparse.hstack((v_fit[:num_train,:],X_train),dtype=float).tocsr()
X_test=sparse.hstack((v_fit[num_train:,:],X_test),dtype=float).tocsr()
train_data = lgb.Dataset(X_train,y_train)
train_data.save_binary('CountVectorizer_train_age.bin')

lgb_params = {'num_leaves': 2**6-1,
              'min_data_in_leaf': 25, 
              'objective':'multiclass',
              'num_class':11,
              'max_depth': -1,
              'learning_rate': 0.1,
              'boosting': 'gbdt',
              'feature_fraction': 0.6,
              'bagging_fraction': 0.9,
              'bagging_seed': 11,
              'metric': 'multi_error',
              'seed':1024,
              'nthread':12,
              'lambda_l1': 0.2,
             }
print("start training")
cv_results = lgb.cv(
    lgb_params, train_data, num_boost_round=1000, nfold=5, stratified=False, shuffle=True,
    early_stopping_rounds=50, verbose_eval=10, show_stdv=True, seed=0)

start training
[10]	cv_agg's multi_error: 0.773665 + 0.00104111
[20]	cv_agg's multi_error: 0.7718 + 0.00112992
[30]	cv_agg's multi_error: 0.769813 + 0.00108029
[40]	cv_agg's multi_error: 0.767744 + 0.00119807
[50]	cv_agg's multi_error: 0.766378 + 0.00120966
[60]	cv_agg's multi_error: 0.764989 + 0.00114233
[70]	cv_agg's multi_error: 0.763919 + 0.00123168
[80]	cv_agg's multi_error: 0.763035 + 0.00112768


In [13]:
# predict age
%reset -f
import lightgbm as lgb
train_data = lgb.Dataset('CountVectorizer_train_age.bin')
lgb_params = {'num_leaves': 2**6-1,
              'min_data_in_leaf': 25, 
              'objective':'multiclass',
              'num_class':11,
              'max_depth': -1,
              'learning_rate': 0.1,
              'boosting': 'gbdt',
              'feature_fraction': 0.6,
              'bagging_fraction': 0.9,
              'bagging_seed': 11,
              'metric': 'multi_error',
              'seed':1024,
              'nthread':12,
              'lambda_l1': 0.2,
             }
print("start training")
model = lgb.train(lgb_params, train_data,num_boost_round=200)
model.save_model("age_model.txt")
print("training completed!")

start training


KeyboardInterrupt: 

In [11]:
# predict age
%reset -f
import pandas as pd
from scipy import sparse
import lightgbm as lgb
from sklearn import metrics
user_info=pd.read_csv('user_info.csv')
user_info.astype(int)
agg=pd.read_csv('agg.csv')
v_fit=sparse.load_npz('v_fit.npz')
num_train=int(user_info.shape[0]*0.8)
y_test=user_info["age"].iloc[num_train:]
display(y_test)
X_test=sparse.csr_matrix(agg.iloc[num_train:])
X_test=sparse.hstack((v_fit[num_train:,:],X_test),dtype=float).tocsr()
print("start predicting")
model=lgb.Booster(model_file='age_model.txt')
y_pred=model.predict(X_test)
display(y_pred)
print("acc:",metrics.accuracy_score(y_test,y_pred))
print("acc:",metrics.roc_auc_score(y_test,y_pred))

720000    3
720001    2
720002    2
720003    3
720004    5
         ..
899995    5
899996    3
899997    4
899998    3
899999    3
Name: age, Length: 180000, dtype: int64

start predicting


array([3.74877228, 4.60749918, 4.16929228, ..., 4.28402148, 4.44051075,
       4.15519538])

ValueError: Classification metrics can't handle a mix of multiclass and continuous targets

In [ ]:
y_pred=

In [6]:
df=pd.DataFrame(y_pred)

In [8]:
df.describe()

,0
count,180000.0
mean,1.0
std,0.0
min,1.0
25%,1.0
50%,1.0
75%,1.0
max,1.0


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
df=tmpdf
features=["creative_id"]
v = TfidfVectorizer(max_df=1.0,min_df=1,lowercase=False,tokenizer=lambda x:x)
for feature in features:
    cur=df.groupby("user_id")[feature].apply(list)
    v_fit = v.fit_transform(cur)
    word=v.get_feature_names()
    display(v_fit)
    display(type(v_fit))
#     for i in range(len(weight)):
#         print("-----output feature={}, user_id={} tf-idf-----".format(feature,i+1))
#         for j in range(len(word)):
#             print(word[j],weight[i][j])